### Do preprocessing

In [3]:
import preprocess

train_df, dev_df, dev_baseline_df, test_df, evidence_df = preprocess.return_df()
original_train_df, original_dev_df, original_dev_baseline_df, original_test_df, original_evidence_df = preprocess.return_original_df()

In [4]:
# print head of the dataframes
print(train_df.head())
print(dev_df.head())
print(dev_baseline_df.head())
print(test_df.head())
print(evidence_df.head())
print("#####################################")
print(original_train_df.head())
print(original_dev_df.head())
print(original_dev_baseline_df.head())
print(original_test_df.head())
print(original_evidence_df.head())

                                                   claim_text  \
claim-1937  [scientific, evidence, co2, pollutant, higher,...   
claim-126   [el, nio, drove, record, high, global, tempera...   
claim-2510                 [1946, pdo, switched, cool, phase]   
claim-2021  [weather, channel, cofounder, john, coleman, p...   
claim-2449  [january, 2008, capped, 12, month, period, glo...   

                claim_label                                          evidences  
claim-1937         DISPUTED  [evidence-442946, evidence-1194317, evidence-1...  
claim-126           REFUTES                [evidence-338219, evidence-1127398]  
claim-2510         SUPPORTS                 [evidence-530063, evidence-984887]  
claim-2021         DISPUTED  [evidence-1177431, evidence-782448, evidence-5...  
claim-2449  NOT_ENOUGH_INFO  [evidence-1010750, evidence-91661, evidence-72...  
                                                   claim_text  \
claim-752   [south, australia, expensive, electricity, wor